In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "Spark")

In [190]:
datos = sc.textFile("ej1.csv").map(lambda x: x.split(','))
datos.take(5)

[['10042', 'Quimica', '5', '02/17/2018'],
 ['10074', 'Algo1', '1', '12/15/2017'],
 ['10099', 'Analisis II', '2', '12/02/2018'],
 ['10024', 'Algo2', '7', '08/15/2017'],
 ['10098', 'Taller de programacion', '6', '05/20/2017']]

In [191]:
#Ej 1 a
aprobados = datos.filter(lambda x : int(x[2])>=4)
anios = aprobados.map(lambda x : (x[0], x[3][-4:]))
anios.reduceByKey(max).filter(lambda x : x[1] >= '2018').count()

101

In [192]:
#Ej 1 b
promedio = datos.map(lambda x : (x[0], (int(x[2]), 1)))
promedio = promedio.reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1]))
promedio = promedio.mapValues(lambda x : x[0]/x[1])
promedio.take(5)

[('10042', 4.7272727272727275),
 ('10074', 5.5),
 ('10099', 4.090909090909091),
 ('10024', 6.111111111111111),
 ('10098', 5.076923076923077)]

In [193]:
#Ej 1 c
mejor = promedio.reduce(lambda x,y : x if x[1]>=y[1] else y)
mejor

('10069', 8.5)

Ejercicio 2:

In [194]:
ventas = sc.textFile("ej2.csv").map(lambda x: x.split(','))
prod = sc.textFile("ej2_productos.csv").map(lambda x: x.split(','))

In [195]:
ventas.take(5)

[['11/04/2019', '20', '338'],
 ['12/09/2019', '1', '412'],
 ['07/10/2019', '61', '395'],
 ['09/26/2019', '94', '52'],
 ['11/03/2019', '88', '216']]

In [196]:
prod.take(5)

[['1', 'quis tortor id nulla', 'Computacion'],
 ['2', 'et tempus semper est quam', 'Comida'],
 ['3', 'sed tristique in tempus sit amet sem', 'Computacion'],
 ['4',
  'nulla justo aliquam quis turpis eget elit sodales scelerisque',
  'Computacion'],
 ['5', 'interdum venenatis turpis enim', 'Comida']]

In [197]:
#Ej 2 a
masVendido = ventas.map(lambda x : (x[1], 1))
masVendido = masVendido.reduceByKey(lambda x, y: x+y)
aux = masVendido.reduce(lambda x,y : x if x[1] > y[1] else y)
aux

('71', 17)

In [210]:
#Ej 2 b
aux = prod.map(lambda x : (x[0], x[2]))
join = aux.join(masVendido)
join = join.map(lambda x : (x[1][0], x[1][1]))
categorias = join.reduceByKey(lambda x,y : x+y)
masVend = categorias.reduce(max)
masVend

('Utileria', 311)

In [275]:
#Ej 2 c
aux = prod.map(lambda x : (x[0], x[1]))
top = aux.join(masVendido)
top = top.map(lambda x : (x[1][1], (x[0], x[1][0])))
top.takeOrdered(5,lambda x: -1*x[0])

[(17, ('71', 'nam congue risus semper porta volutpat quam pede')),
 (16, ('65', 'id pretium iaculis diam')),
 (15, ('17', 'mauris eget massa tempor')),
 (15, ('68', 'diam vitae quam suspendisse potenti nullam porttitor lacus at')),
 (15,
  ('77', 'est lacinia nisi venenatis tristique fusce congue diam id ornare'))]

In [293]:
#Ej 2 d
ultAnio = ventas.map(lambda x : (x[0][-4:], (x[2], x[1])))
ultAnio = ultAnio.filter(lambda x : x[0] == '2019')
masDe10 = ultAnio.map(lambda x : (x[1][1], 1)).reduceByKey(lambda x,y : x+y).filter(lambda x : x[1] >= 10).collect()
aux = [i[0] for i in masDe10]
ultAnio = ultAnio.filter(lambda x : x[1][1] in aux)
masAumento = ultAnio.map(lambda x : (x[1][1], int(x[1][0])))
maximo = masAumento.reduceByKey(max)
minimo = masAumento.reduceByKey(min)
maxMin = maximo.join(minimo)
aumento = maxMin.map(lambda x : (x[0], x[1][0] - x[1][1]))
masAumento = aumento.reduce(lambda x, y : x if x[1] > y[1] else y)
masAumento

('56', 442)